In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv


In [2]:
import pandas as pd
sample_submission = pd.read_csv("../input/ieee-fraud-detection/sample_submission.csv")
test_identity = pd.read_csv("../input/ieee-fraud-detection/test_identity.csv")
test_transaction = pd.read_csv("../input/ieee-fraud-detection/test_transaction.csv")
train_identity = pd.read_csv("../input/ieee-fraud-detection/train_identity.csv")
train_transaction = pd.read_csv("../input/ieee-fraud-detection/train_transaction.csv")

In [3]:
train = pd.concat([train_identity.iloc[:,0:6], train_transaction.iloc[:,0:6]], axis=1)
test = pd.concat([test_identity.iloc[:,0:6], test_transaction.iloc[:,0:6]], axis=1)

In [4]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer

In [5]:
X = train.drop(['isFraud'],axis=1)
y = train.isFraud

In [6]:
X.ProductCD.fillna(value='most_frequent').value_counts()
X.ProductCD.isnull().any().sum()

0

In [7]:
X_num = X.select_dtypes(exclude='object')
list(X_num)

['TransactionID',
 'id_01',
 'id_02',
 'id_03',
 'id_04',
 'id_05',
 'TransactionID',
 'TransactionDT',
 'TransactionAmt',
 'card1']

In [8]:
col_trans = make_column_transformer((list(X_num), SimpleImputer(strategy='mean')),
                              (list(X_num), StandardScaler()),
                              (['ProductCD'],OneHotEncoder(sparse=False)))

/opt/conda/lib/python3.6/site-packages/sklearn/compose/_column_transformer.py:778: DeprecationWarning: `make_column_transformer` now expects (transformer, columns) as input tuples instead of (columns, transformer). This has been introduced in v0.20.1. `make_column_transformer` will stop accepting the deprecated (columns, transformer) order in v0.22.
  warnings.warn(message, DeprecationWarning)


In [9]:
col_trans.fit_transform(X)

array([[ 2.98700400e+06,  2.98700000e+06,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 2.98700800e+06,  2.98700100e+06, -5.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 2.98701000e+06,  2.98700200e+06, -5.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       ...,
       [ 3.23632931e+06,  3.57753700e+06, -1.01705019e+01, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 3.23632931e+06,  3.57753800e+06, -1.01705019e+01, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 3.23632931e+06,  3.57753900e+06, -1.01705019e+01, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [10]:
pipe_cat = make_pipeline(OneHotEncoder(sparse=False))
pipe_num = make_pipeline(SimpleImputer(strategy='most_frequent'),(StandardScaler()))
                         
col_trans2 = make_column_transformer((list(X_num), pipe_num),
                                     (['ProductCD'], pipe_cat), remainder='passthrough')

/opt/conda/lib/python3.6/site-packages/sklearn/compose/_column_transformer.py:778: DeprecationWarning: `make_column_transformer` now expects (transformer, columns) as input tuples instead of (columns, transformer). This has been introduced in v0.20.1. `make_column_transformer` will stop accepting the deprecated (columns, transformer) order in v0.22.
  warnings.warn(message, DeprecationWarning)


In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

model = make_pipeline(col_trans2, LogisticRegression())

In [12]:
model.fit(X,y)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='most_frequent',
                             

In [13]:
model.score(X,y)

0.9650099908558268

In [14]:
scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

In [15]:
scores

array([0.9650069 , 0.9650069 , 0.9650066 , 0.96501477, 0.96501477])

In [16]:
pred = model.predict_proba(test)
prob = pred[:,1]

In [17]:
sample_submission

,TransactionID,isFraud
0,3663549,0.5
1,3663550,0.5
2,3663551,0.5
3,3663552,0.5
4,3663553,0.5
...,...,...
506686,4170235,0.5
506687,4170236,0.5
506688,4170237,0.5
506689,4170238,0.5


In [18]:
prob.shape
trid = test.TransactionID.iloc[:,1]
trid

0         3663549
1         3663550
2         3663551
3         3663552
4         3663553
           ...   
506686    4170235
506687    4170236
506688    4170237
506689    4170238
506690    4170239
Name: TransactionID, Length: 506691, dtype: int64

In [19]:
submission = pd.DataFrame({'TransactionID':trid, 'isFraud':pred[:,1]})

In [20]:
submission

,TransactionID,isFraud
0,3663549,0.024050
1,3663550,0.024050
2,3663551,0.024050
3,3663552,0.024050
4,3663553,0.024050
...,...,...
506686,4170235,0.014551
506687,4170236,0.014551
506688,4170237,0.014551
506689,4170238,0.014551


In [21]:
filename = 'Fraud Detection.csv'

submission.to_csv(filename,index=False)